In [1]:
import OffsetArrays
using Plots
using Revise
using StatsBase
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using Distributions
srcpath = "./src"
include("$srcpath/bp.jl") #julia code which contains the bp update
include("$srcpath/post_infer.jl") #definition of the main functions and the types
include("$srcpath/observables.jl") # julia code for estimating observables

avg_ninf (generic function with 1 method)

In [2]:
λp = 1.0# planted infection rate
λi = λp #inferred infection rate

T = 8 # discrete time: number of time-steps

γp = 0.1 #0.019 # planted autoinfection probability
γi = γp # inferred autoinfection probability

N = 5000; #population size
@show 2/sqrt(N)

dilution = 0.0 #dilution of observations. dil=0 means everybody observed once
fr = 0.0; #noise in the observation. 

degree_dist = Dirac(3) #the distribution of the degree in the graph. Dirac means random regular.

2 / sqrt(N) = 0.0282842712474619


Dirac{Int64}(value=3)

In [3]:
M_zero = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [4]:
@time sweep!(M_zero)

  2.607082 seconds (25.21 M allocations: 1.620 GiB, 9.24% gc time, 64.49% compilation time)


0.06993729216328648

In [5]:
F_zero,it_zero = pop_dynamics(M_zero, tot_iterations = 20);

#1.iter 2.err 3.F
0	-1	-0.4709943938605553
1	-1	-0.395455570483116
2	-1	-0.3910154902316936
3	-1	-0.3910143954716972
4	-1	-0.39101439547165867
5	-1	-0.39101439547165867
6	-1	-0.39101439547165867
7	-1	-0.3910143954716579
8	-1	-0.39101439547165867
9	-1	-0.39101439547165867
10	-1	-0.39101439547165867
11	3.284599251323509e-7	-0.39101439547165867
12	0.0	-0.39101439547165867
13	0.0	-0.39101439547165867
14	0.0	-0.39101439547165867
15	0.0	-0.39101439547165867
16	0.0	-0.39101439547165867
17	0.0	-0.39101439547165867
18	0.0	-0.39101439547165867
19	0.0	-0.39101439547165867


In [6]:
M_zero.belief[:,:,186]

10×10 OffsetArray(::Matrix{Float64}, 0:9, 0:9) with eltype Float64 with indices 0:9×0:9:
 0.0  0.0  0.1          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.2439       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.307422     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.250201     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0906219    0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.00780519   0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  4.99779e-5   0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  2.02338e-9   0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  3.31618e-18  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [7]:
#M_zero.mom1μ
#sum(M_zero.mom1μ)./length(M_zero.mom1μ)

In [8]:
#marg_zero = M_zero.belief |> real;
#marg2D_zero = reshape(sum(marg_zero,dims=3) ./ N, T+2,T+2)
#ensAUC_zero = avgAUC(marg_zero,M_zero.obs_list,count_obs=true);

In [9]:
M = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [10]:
F,it = pop_dynamics_stab(M, tot_iterations = 100, nbstab=40);

#1.iter 2.err 3.F 4. Δ
0	-1	0.39936231795824567	-1.0
1	-1	0.15627128685712524	-1.0
2	-1	-0.3522791615790455	-1.0
3	-1	-0.490256349086055	-1.0
4	-1	-0.4866855067623772	-1.0
5	-1	-0.4331438261549803	-1.0
6	-1	-0.4017927920140915	-1.0
7	-1	-0.3925434516612935	-1.0
8	-1	-0.3911156828081017	-1.0
9	-1	-0.3910176285721842	-1.0
10	-1	-0.3910144339858889	-1.0
11	0.042373924834874545	-0.39101439559499923	-1.0
12	0.03830480754548754	-0.39101439547168554	-1.0
13	0.02964791973883933	-0.39101439547165867	-1.0
14	0.012633028731259585	-0.39101439547165867	-1.0
15	0.0032111728500158914	-0.39101439547165867	-1.0
16	0.0004562250022193074	-0.39101439547165867	-1.0
17	3.0292438980499958e-5	-0.39101439547165867	-1.0
18	9.68690095135272e-7	-0.39101439547165867	-1.0
19	0.0	-0.39101439547165867	-1.0
20	0.0	-0.39101439547165867	-1.0
21	0.0	-0.39101439547165867	-1.0
22	0.0	-0.39101439547165867	-1.0
23	0.0	-0.39101439547165867	-1.0
24	0.0	-0.39101439547165867	-1.0
25	0.0	-0.39101439547165867	-1.0
26	0.0	-0.391014

In [12]:
@time sweep_stab!(M, 3, 4)

  0.966316 seconds (9.55 M allocations: 904.584 MiB, 8.06% gc time)


(-0.39101439547165867, 4.243443129635428e-104)

In [16]:
test = rand(-1:2:1, T+2, 2, T+2, 3, N).*0.00001;
mean(test), mean(M.μ), mean(M.mom1μ)

(6.0866666666666634e-9, 0.0016666666666666668, 4.324331667368392e-9)

In [17]:
extrema(test), extrema(M.μ), extrema(M.mom1μ)

((-1.0e-5, 1.0e-5), (0.0, 0.08090472429788192), (-1.334582163922407e-7, 4.109041142782289e-7))

In [18]:
std(test), std(M.μ), std(M.mom1μ)

(9.999999814291046e-6, 0.0029692850196844913, 1.1095832860748813e-8)

(-7.746640579475731e-16, 2.0479244851529683e-15)

In [ ]:
marg = M.belief |> real;
marg2D = reshape(sum(marg,dims=3) ./ N, T+2,T+2)
ensAUC = avgAUC(marg,M.obs_list,count_obs=true); #set false to see the AUC only on NON observed indiv.

In [ ]:
sum(abs.(marg2D.-marg2D_zero))/length(marg2D)

In [ ]:
scatter(ensAUC)
scatter!(ensAUC_zero)

In [ ]:
pheat = heatmap((marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred",title="Average 2D marginal")
pheat_stab = heatmap((marg2D_zero.-marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred")
plot(pheat,pheat_stab)

In [ ]:
scatter(F)
scatter!(F_zero)
hline!([0])